# Signal-to-Noise Ratio (SNR)

The snr module contains four functions for calculating the SNR depending on whether a binary is stationary/evolving and circular/eccentric using various approximations. All you have to do is set the gw_lum_tol and stat_tol when instantiating the class and then call legwork.source.Source.get_snr().

This splits the sources into (up to) 4 subsets and calculates their SNRs before recollecting them and storing the result in source.snr.

Let’s try this out. Note that we run with verbose=True here so that you can see the size of subpopulation.

# Demo - Basic SNR Calculation

This demo shows how to compute the SNR of a single binary system, as well as a collection of systems.

In [2]:
import GravitationalWaves as gw
import astropy.units as u


## Single source SNR calculation

First, we calculate the signal-to-noise ratio for a single stellar-mass binary system. Let’s create a toy source and calculate its SNR (for a default 4-year LISA mission).

In [4]:
source = gw.source.Source(m_1 = 10 * u.Msun,
                          m_2 = 10 * u.Msun,
                          ecc = 0.2,
                          f_orb = 1e-4 * u.Hz,
                          dist = 8 * u.kpc,
                          interpolate_g = False)
source.get_snr(verbose = True)

Calculating SNR for 1 sources
	0 sources have already merged
	1 sources are stationary
		1 sources are stationary and eccentric


array([4.49425701])

The package checked whether the source is eccentric/circular and evolving/stationary and picked the fastest possible way to calculate the SNR accurately.



## Population of sources SNR calculation

If we want to know the SNR of three (or any number of) sources then you can instead provide arrays for each of the arguments and execute the code in exactly the same way.

In [11]:
# supply arrays that are identical to earlier example but different primary masses
sources = gw.source.Source(m_1=[5, 10, 20] * u.Msun,
                           m_2=[10, 10, 10] * u.Msun,
                           ecc=[0.2, 0.2, 0.2],
                           f_orb=[1e-4, 1e-4, 1e-4] * u.Hz,
                           dist=[8, 8, 8] * u.kpc,
                           interpolate_g=False)

sources.get_snr(verbose = True).round(2)

Calculating SNR for 3 sources
	0 sources have already merged
	3 sources are stationary
		3 sources are stationary and eccentric


array([2.47, 4.49, 7.85])

----------------------------------------

We can also adjust the length of the LISA mission to see how this affects the SNR.



In [1]:
from GravitationalWaves import source, utils, visualization
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

ModuleNotFoundError: No module named 'GravitationalWaves'

In [29]:
# create a random collection of sources
n_values = 1500
m_1 = np.random.uniform(0, 10, n_values) * u.Msun
m_2 = np.random.uniform(0, 10, n_values) * u.Msun
dist = np.random.normal(8, 1.5, n_values) * u.kpc
f_orb = 10**(-5 * np.random.power(3, n_values)) * u.Hz
ecc = 1 - np.random.power(3, n_values)

In [24]:
sources = source.Source(m_1=m_1, m_2=m_2, ecc=ecc, dist=dist, f_orb=f_orb, gw_lum_tol=0.05, stat_tol=1e-2,
                        interpolate_g=True, interpolate_sc=True,
                        sc_params={
                            "instrument": "LISA",
                            "custom_psd": None,
                            "t_obs": 4 * u.yr,
                            "L": 2.5e9 * u.m,
                            "approximate_R": False,
                            "confusion_noise": 'robson19'
                        })

In [32]:
snr_4 = sources.get_snr(verbose=True)
snr_10 = sources.get_snr(t_obs=10 * u.yr, verbose=True)

Calculating SNR for 1500 sources
	0 sources have already merged
	1347 sources are stationary
		263 sources are stationary and circular
		1084 sources are stationary and eccentric
	153 sources are evolving
		26 sources are evolving and circular
		127 sources are evolving and eccentric
Calculating SNR for 1500 sources
	0 sources have already merged
	1347 sources are stationary
		263 sources are stationary and circular
		1084 sources are stationary and eccentric
	153 sources are evolving
		26 sources are evolving and circular
		127 sources are evolving and eccentric


Note that you can see that the number of stationary binaries has decreased slightly since some binaries may just be on the cusp of no longer being stationary and extending the time means they change frequency enough to be labelled as evolving. It could be interesting to do this to see how the number of detectable binaries changes.



In [27]:
n_detect_4 = len(snr_4[snr_4 > 7])
n_detect_10 = len(snr_10[snr_10 > 7])
print("{} binaries are detectable over 4 years".format(n_detect_4))
print("Whilst extending to a 10 year mission gives {}".format(n_detect_10))

626 binaries are detectable over 4 years
Whilst extending to a 10 year mission gives 661
